# Review NCBI Yellow Fever (YF) Sequences and SimReads Data

In this notebook, we review the yellow fever sequences retrieved from NCBI GenBank and the corresponding simulated reads generated with ART Illumina.

> Note: 
>
> The notebook was built and tested to run locally but should also work on Colab or Kaggle. If on Colab, it assumes that the project shared gdrive directory is accessible through a shortcut called `Metagenomics` under the root of gdrive.

# 1. Imports and setup environment

### Install and import packages

In [ ]:
# Install required custom packages if not installed yet.
import importlib.util
if not importlib.util.find_spec('eccore'):
    print('installing package: `eccore`')
    ! pip install -qqU eccore
else:
    print('`eccore` already installed')
if not importlib.util.find_spec('metagentorch'):
    print('installing package: `metagentorch')
    ! pip install -qqU metagentorch
else:
    print('`metagentorch` already installed')

`eccore` already installed
`metagentorch` already installed


In [ ]:
# Import all required packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from eccore.core import files_in_tree
from eccore.ipython import nb_setup
from pathlib import Path
from pprint import pprint

# Setup the notebook for development
nb_setup()

from metagentorch.cnn_virus.data import FastaFileReader, FastqFileReader, AlnFileReader
from metagentorch.core import ProjectFileSystem

Set autoreload mode


# 2. Project file system

This project adopts a unified file structure to make coding and colaboration easier. In addition, we can run the code locally (from a `project-root` directory) or in the cloud (colab or kaggle).

The unified file structure when running localy is:
```text
    project-root   
        |--- data
        |      |--- CNN_Virus_data  (all data related to CNN Virus original paper)
        |      |--- ncbi            (all data related to NCBI reference sequences)
        |      |--- saved           (trained and finetuned models)
        |      |--- ....            (raw and pre-processed data from various sources)  
        |      
        |--- nbs  (all reference and work notebooks)
        |      |--- cnn_virus
        |      |        |--- ref_n.n_notebooks.ipynb
        |      |        |--- wip_n.n_notebooks.ipynb
```

When running on google colab, it is assumed that a google drive is mounted on the colab server instance, and that the google drive root includes a shortcut named `Metagenomics` and pointing to the project shared directory. The project shared directory is accessible [here](/https://drive.google.com/drive/folders/134uei5fmt08TpzhmjG4sW0FQ06kn2ZfZ) if your google account was given access permission.

To make access to the unified file system, `metagentools` provides a `ProjectFileSystem` class (see [documentation](https://vtecftwy.github.io/metagentools/core.html#projectfilesystem) for more info).

Key folders and system information

In [ ]:
pfs = ProjectFileSystem()
pfs.info()

Running linux on local computer
Device's home directory: /home/vtec
Project file structure:
 - Root ........ /home/vtec/projects/bio/metagentorch 
 - Data Dir .... /home/vtec/projects/bio/metagentorch/data 
 - Notebooks ... /home/vtec/projects/bio/metagentorch/nbs


In [ ]:
pfs.readme()

ReadMe file for directory `data`:

### Data structure for this project
This directory includes all the data required for the project.

```text
data
 |--- CNN_Virus_data 
 |--- ncbi                
 |--- saved         
 |--- yf-reads
 |--- ....           
     
```
#### Sub-directories
- `CNN_Virus_data`: includes all the data related to the original CNN Virus paper, i.e. training data and validation data in a format that can be used by the CNN Virus code.
- `ncbi`: includes data related to the use of viral sequences from NCBI: reference sequences, simulated reads, inference datasets, inference results.
- `saved`: includes model saved parameters and preprocessing datasets.
- `yf-reads`: includes all data related to real yellow fever reads, from "wet" samples

Also available on AWS S3 at `https://s3.ap-southeast-1.amazonaws.com/bio.cnn-virus.data/data/...`

## Explore files in NCBI data directory

In [ ]:
pfs.readme(dir_path=pfs.data/'ncbi');

ReadMe file for directory `data/ncbi`:

### NCBI Data

This directory includes all data related to the work done with reference sequences from NCBI. 

The data is organized in the following subfolders:

- `refsequences`: reference CoV sequences downloaded from NCBI, and related metadata
- `simreads`: all data from simulated reads, using ART Illumina simulator and the reference sequences
- `infer_results`: results from the inference using models with the simulated reads
- `ds`: datasets in proper format for training or inference/prediction using the CNN Virus model


In [ ]:
for d in ['refsequences', 'simreads', 'infer_results']:
    path = pfs.data / 'ncbi' / d
    pfs.readme(dir_path=path)
    # print(f"{'-'*100}")
    files_in_tree(path)
    print(f"{'='*100}")

ReadMe file for directory `data/ncbi/refsequences`:

### NCBI Reference Sequences

This directory includes the reference sequences from NCBI, for each virus group. 

Each group of viruses is grouped into one directory:
- `cov`: all reference sequences and related metadata files for corona (sars and mers)
- `mRhiFer1`:  a reference sequence different from CoV and YF to test false positive predictions
- `yf`: all reference sequences and related metadata files for yellow fever


ncbi
  |--refsequences
  |    |--readme.md (0)
  |    |--mRhiFer1
  |    |    |--Rhinolophus_ferrumequinum.mRhiFer1_v1.p.dna_rm.primary_assembly.1.fa (1)
  |    |    |--Rhinolophus_ferrumequinum.mRhiFer1_v1.p.dna.primary_assembly.1.fa.gz (2)
  |    |    |--Rhinolophus_ferrumequinum.mRhiFer1_v1.p.dna_rm.primary_assembly.1.clean.fa (3)
  |    |    |--readme.md (4)
  |    |    |--Rhinolophus_ferrumequinum.mRhiFer1_v1.p.dna_rm.primary_assembly.1.fa.gz (5)
  |    |--yf
  |    |    |--yf_1971_angola.fa (6)
  |    |    |--yf_2023_yellow_fever.fa (7)
  |    |    |--yf_2023_yellow_fever_with_missing_bases.fa (8)
  |    |    |--yf_2023_yellow_fever_aligned.fa (9)
  |    |    |--yf_2023_multiple_alignment_original.fa (10)
  |    |    |--readme.md (11)
  |    |    |--yf_NC_002031_full_sequence.fa (12)
  |    |--cov
  |    |    |--cov_refseq_original.fa (13)
  |    |    |--cov_original_cnn_sequences.json (14)
  |    |    |--cov_refseq_002-seqs.fa (15)
  |    |    |--cov_refseq_025-seqs.fa (16)
  | 

ReadMe file for directory `data/ncbi/simreads`:

### NCBI simulated reads
This directory includes all sets of simulated read sequence files generated from NCBI viral sequences using  ARC Illumina. 

```ascii
this-directory
    |--cov
    |    |
    |    |--single_10seq_50bp
    |    |    |--single_10seq_50bp.fq
    |    |    |--single_10seq_50bp.alnEnd
    |    |-- ...
    |    |--single_100seq_150bp
    |    |    |--single_100seq_150bp.fq
    |    |    |--single_100seq_150bp.aln
    |    |--paired_100seq_50bp
    |    |    |--paired_100seq_50bp2.aln
    |    |    |--paired_100seq_50bp1.aln
    |    |    |--paired_100seq_50bp2.fq
    |    |    |--paired_100seq_50bp1.fq
    |    |-- ...
    |    |
    |---yf
    |    |
    |    |--yf_AY968064-single-150bp
    |    |    |--yf_AY968064-single-1seq-150bp.fq
    |    |    |--yf_AY968064-single-1seq-150bp.aln
    |    |
    |--mRhiFer1
    |    |--mRhiFer1_v1.p.dna_rm.primary_assembly.1
    |    |    |--mRhiFer1_v1.p.dna_rm.primary_assembly.1.fq
    |    |    |--mRhiFer1_v1.p.dna_rm.primary_assembly.1.aln
    |    |

```

This directory includes several subdirectories, each for one virus, e.g. `cov` for corona, `yf` for yellow fever.

In each virus subdirectory, several simreads directory includes simulated reads with various parameters, named as `<method>_<nb-seq>_<nb-bp>` where"
- `<method>` is either `single` or `paired` depending on the simulation method
- `<nb-seq>` is the number of reference sequences used for simulation, and refers to the `fa` file used
- `<nb-bp>` is the number of base pairs used to simulate reads


Each sub-directory includes simreads files made using a simulation method and a specific number of reference sequences.
- `xxx.fq` and `xxx.aln` files when method is `single`
- `xxx1.fq`, `xxx2.fq`, `xxx1.aln` and `xxx2.aln` files when method is `paired`.

Example:
- `paired_10seq_50bp` means that the simreads were generated by using the `paired` method to simulate 50-bp reads, and using the `fa` file `cov_virus_sequences_010-seqs.fa`.
- `single_100seq_50bp` means that the simreads were generated by using the `single` method to simulate 50-bp reads, and using the `fa` file `cov_virus_sequences_100-seqs.fa`. Note that this generated 20,660,104 reads !

#### Simread file formats

Simulated reads information is split between two files:
- **FASTQ** (`.fq`) files providing the read sequences and their ASCII quality scores
- **ALN** (`.aln`) files with alignment information

##### FASTQ (`.fq`)
FASTQ files generated by ART Illumina have the following structure (showing 5 reads), with 4 lines for each read:

```ascii
@2591237:ncbi:1-60400
ACAACTCCTATTCGTAGTTGAAGTTGTTGACAAATACTTTGATTGTTACG
+
CCCBCGFGBGGGGGGGBGGGGGGGGG>GGG1G=/GGGGGGGGGGGGGGGG
@2591237:ncbi:1-60399
GATCAATGTGGCATCTACAATACAGACAGCATGAAGCACCACCAAAGGAC
+
BCBCCFGGGGGGGG1CGGGG<GGBGGGGGFGCGGGGGGDGGG/GG1GGGG
@2591237:ncbi:1-60398
ATCTACCAGTGGTAGATGGGTTCTTAATAATGAACATTATAGAGCTCTAC
+
CCCCCGGGEGG1GGF1G/GGEGGGGGGGGGGGGFFGGGGGGGGGGDGGDG
@2591237:ncbi:1-60397
CGTAAAGTAGAGGCTGTATGGTAGCTAGCACAAATGCCAGCACCAATAGG
+
BCCCCGGGFGGGGGGFGGGGFGG1GGGGGGG>GG1GGGGGGGGGGE<GGG
@2591237:ncbi:1-60396
GGTATCGGGTATCTCCTGCATCAATGCAAGGTCTTACAAAGATAAATACT
+
CBCCCGGG@CGGGGGGGGGGGG=GFGGGGDGGGFG1GGGGGGGG@GGGGG
```
The following information can be parsed from the each read sequence in the FASTQ file:

- Line 1: `readid`, a unique ID for the read, under for format `@readid` 
- Line 2: `readseq`, the sequence of the read
- Line 3: a separator `+`
- Line 4: `read_qscores`, the base quality scores encoded in ASCII 

Example:
```
@2591237:ncbi:1-60400
ACAACTCCTATTCGTAGTTGAAGTTGTTGACAAATACTTTGATTGTTACG
+
CCCBCGFGBGGGGGGGBGGGGGGGGG>GGG1G=/GGGGGGGGGGGGGGGG
```
- `readid` = `2591237:ncbi:1-60400`
- `readseq` = `ACAACTCCTATTCGTAGTTGAAGTTGTTGACAAATACTTTGATTGTTACG`, a 50 bp read
- `read_qscores` = `CCCBCGFGBGGGGGGGBGGGGGGGGG>GGG1G=/GGGGGGGGGGGGGGGG`


#### ALN (`.aln`) 
ALN files generated by ART Illumina consist of :
- a header with the ART-Ilumina command used for the simulation (`@CM`) and info on each of the reference sequences used for the simulations (`@SQ`). Header always starts with `##ART_Illumina` and ends with `##Header End` :
- the body with 3 lines for each read:
    1. definition line with `readid`, 
        - reference sequence identification number `refseqid`, 
        - the position in the read in the reference sequence `aln_start_pos` 
        - the strand the read was taken from `ref_seq_strand`. `+` for coding strand and `-` for template strand
    2. aligned reference sequence, that is the sequence segment in the original reference corresponding to the read
    3. aligned read sequence, that is the simmulated read sequence, where each bp corresponds to the reference sequence bp in the same position.

Example of a ALN file generated by ART Illumina (showing 5 reads):

```ascii
##ART_Illumina    read_length    50
@CM    /bin/art_illumina -i /home/vtec/projects/bio/metagentools/data/cov_data/cov_virus_sequences_ten.fa -ss HS25 -l 50 -f 100 -o /home/vtec/projects/bio/metagentools/data/cov_simreads/single_10seq_50bp/single_10seq_50bp -rs 1674660835
@SQ    2591237:ncbi:1 1   MK211378    2591237    ncbi    1     Coronavirus BtRs-BetaCoV/YN2018D    30213
@SQ    11128:ncbi:2   2   LC494191    11128    ncbi    2     Bovine coronavirus    30942
@SQ    31631:ncbi:3   3   KY967361    31631    ncbi    3     Human coronavirus OC43        30661
@SQ    277944:ncbi:4  4   LC654455    277944    ncbi    4     Human coronavirus NL63    27516
@SQ    11120:ncbi:5   5   MN987231    11120    ncbi    5     Infectious bronchitis virus    27617
@SQ    28295:ncbi:6   6   KU893866    28295    ncbi    6     Porcine epidemic diarrhea virus    28043
@SQ    28295:ncbi:7   7   KJ645638    28295    ncbi    7     Porcine epidemic diarrhea virus    27998
@SQ    28295:ncbi:8   8   KJ645678    28295    ncbi    8     Porcine epidemic diarrhea virus    27998
@SQ    28295:ncbi:9   9   KR873434    28295    ncbi    9     Porcine epidemic diarrhea virus    28038
@SQ    1699095:ncbi:10 10  KT368904    1699095    ncbi    10     Camel alphacoronavirus    27395
##Header End
>2591237:ncbi:1    2591237:ncbi:1-60400    14770    +
ACAACTCCTATTCGTAGTTGAAGTTGTTGACAAATACTTTGATTGTTACG
ACAACTCCTATTCGTAGTTGAAGTTGTTGACAAATACTTTGATTGTTACG
>2591237:ncbi:1    2591237:ncbi:1-60399    17012    -
GATCAATGTGGCATCTACAATACAGACAGCATGAAGCACCACCAAAGGAC
GATCAATGTGGCATCTACAATACAGACAGCATGAAGCACCACCAAAGGAC
>2591237:ncbi:1    2591237:ncbi:1-60398    9188    +
ATCTACCAGTGGTAGATGGGTTCTTAATAATGAACATTATAGAGCTCTAC
ATCTACCAGTGGTAGATGGGTTCTTAATAATGAACATTATAGAGCTCTAC
.....
```

ncbi
  |--simreads
  |    |--readme.md (0)
  |    |--mRhiFer1
  |    |    |--readme.md (1)
  |    |--yf
  |    |    |--readme.md (2)
  |    |--cov
  |    |    |--readme.md (3)


ReadMe file for directory `data/ncbi/infer_results`:

#### Inference Result Files

Each directory includes a set of prediction/inference result files.

```shell
ncbi
  |--infer_results
  |    |--readme.md
  |    |--cov-ncbi
  |    |    |--readme.md
  |    |    |--cnn_virus   (directory with results in parquet format)
  |    |    |--csv         (directory with results in csv format)
  |    |    |--xlsx        (directory with results in xlsx format)
  |    |--yf-ncbi
  |    |    |--readme.md
  |    |    |--local-inf   (directory with results per sequence in csv format)
  |    |    |--single_1seq_150bp.db          (inference results in a sqlite database)
  |    |    |--single_1seq_150bp-backup.db   (inference results in a sqlite database)
  |    |    |--selected_test.db              (inference results in a sqlite database)
```

ncbi
  |--infer_results
  |    |--readme.md (0)
  |    |--cov-ncbi
  |    |    |--readme.md (1)
  |    |--yf-ncbi
  |    |    |--readme.md (2)
  |    |    |--single_1seq_150bp.db (3)
  |    |    |--single_1seq_150bp-backup.db (4)
  |    |    |--selected_test.db (5)


# 3. Review fasta, fastq and aln files

## Setup paths

- `p2refseqs`: path to a reference sequence file (FASTA)
- `p2simreads`: path to folder where reads files are located (FASTQ and ALN)

In [ ]:
p2refseqs = pfs.data / 'ncbi/refsequences/yf/yf_2023_yellow_fever.fa'
assert p2refseqs.is_file(), f"No file found at {p2refseqs.absolute()}"

p2simreads = pfs.data / 'ncbi/simreads/yf/single_69seq_150bp'
assert p2simreads.is_dir(), f"No directory found at {p2simreads.absolute()}"

## Explore Reference Files (FASTA)

`FastaFileReader` is a class to make reading and accessing fasta file information easier

In [ ]:
fasta = FastaFileReader(path=p2refseqs)

In [ ]:
fasta.print_first_chunks(nchunks=3)


Sequence 1:
>11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971
ATGTCTGGTCGAAAAGCTCAGGGTAAAACCCTGGGCGTCAATATGGTAAGACGAGGGGTTCGCTCCTTGTCAAACAAAAT ...

Sequence 2:
>11089:ncbi:2	2	U54798	11089	ncbi	Ivory_Coast_1982
ATGTCTGGTCGCAAAGCTCAGGGAAAAACCCTGGGCGTCAATATGGTTCGACGGGGAGTCCGCTCCTTGTCAAACAAAAT ...

Sequence 3:
>11089:ncbi:3	3	DQ235229	11089	ncbi	Ethiopia_1961
ATGTCTGGTCGAAAAGCTCAGGGTAAAACCCTGGGCGTCAATATGGTAAGACGGGGAGCACGCTCCTTGTCAAACAAAAT ...


Access each sequence (definition line and sequence) one by one

In [ ]:
fasta.reset_iterator()

for i, seq in enumerate(fasta):
    print(f"Definition Line for sequence {i+1}:")
    print(seq['definition line'])
    print(f"{len(seq['sequence']):,d} bases:")
    print(seq['sequence'])
    print()
    if i >= 2: break

Definition Line for sequence 1:
>11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971
10,234 bases:
ATGTCTGGTCGAAAAGCTCAGGGTAAAACCCTGGGCGTCAATATGGTAAGACGAGGGGTTCGCTCCTTGTCAAACAAAATAAAACAAAAAACAAAACAGATTGGAAACAGACCTGGCCCTTCAAGAGGTGTTCAAGGATTTATTTTCTTCTTTTTGTTTAACATTCTGACTGGGAAAAAGTTGACTGCTCATCTAAAAAAACTTTGGAGGATGCTTGATCCAAGGCAGGGACTTGCTGTACTGCGGAAGGTCAAAAGAGTTGTAGCTAGCTTAATGAGAGGACTGTCTTCCAGGAAACGTAGATCCAATGAAATGGCCTTGTTTCCACTCTTGTTACTGGGTTTGTTGGCTCTATCAGGAGGAGTGACCCTCGTCAGAAAGAACAGATGGTTGCTCTTGAATGTAACTGCTGAAGATCTGGGGAAAACGTTTTCAGTGGGAACTGGGAATTGCACCACGAATATTCTGGAAGCAAAATACTGGTGCCCCGACTCAATGGAGTACAATTGTCCCAATCTCAGTCCAAGAGAAGAGCCAGATGACATAGATTGCTGGTGTTATGGAGTGGAAAATGTCAGAGTGGCCTATGGAAGATGTGATGCGGTAGGGCGATCAAAACGTTCTAGGAGAGCGATTGATCTACCCACACATGAGAACCATGGACTAAAGACTCGGCAGGAGAAGTGGATGACTGGCAGAATGGGTGAGAGGCAGCTCCAGAAGATTGAAAGATGGCTGGTTAGGAATCCATTTTTTGCGGTCACAGCATTGGCAATAGCCTATCTGGTGGGTAACAACACGACGCAACGAGTGGTAATAGCACTACTGGTTCTAGCGGTTGGTCCAGCGTACTCTGCCCATTGCATAGGGATAACCGACAGAGATTTCATTGAAGGTGTCCATGGG

In [ ]:
definition_lines = fasta.parse_file()
fasta.reset_iterator()

for i, (k, v) in enumerate(definition_lines.items()):
    print(f"Definition Line for sequence {i+1}:")
    print(next(fasta)['definition line'])
    print('Parsed data:')
    print(f"key: {k}\nmetadata:")
    pprint(v)
    print(f"{'='*100}")
    if i >= 2: break

Definition Line for sequence 1:
>11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971
Parsed data:
key: 11089:ncbi:1
metadata:
{'accession': 'AY968064',
 'organism': 'Angola_1971',
 'seqid': '11089:ncbi:1',
 'seqnb': '1',
 'source': 'ncbi',
 'taxonomyid': '11089'}
Definition Line for sequence 2:
>11089:ncbi:2	2	U54798	11089	ncbi	Ivory_Coast_1982
Parsed data:
key: 11089:ncbi:2
metadata:
{'accession': 'U54798',
 'organism': 'Ivory_Coast_1982',
 'seqid': '11089:ncbi:2',
 'seqnb': '2',
 'source': 'ncbi',
 'taxonomyid': '11089'}
Definition Line for sequence 3:
>11089:ncbi:3	3	DQ235229	11089	ncbi	Ethiopia_1961
Parsed data:
key: 11089:ncbi:3
metadata:
{'accession': 'DQ235229',
 'organism': 'Ethiopia_1961',
 'seqid': '11089:ncbi:3',
 'seqnb': '3',
 'source': 'ncbi',
 'taxonomyid': '11089'}


## Explore simulated read output files (FASTQ and ALN)

When simulating with ART Illumina,:
- We simulated reads from referenmce sequences in a fasta file (`fasta` reader)
- ART returns two files:
    - a fastq file (`fastq` reader) with the reads sequences, their quality and some read metadata
    - a aln file (`aln` reader), including:
        - a header with the summulation command and the list of reference sequences
        - additional metadata on each reads, including: 
            - the readid and its number (rank in the list)
            - the read sequence itself (may include errors)
            - the same sequence as in the reference sequence
            - the read alignment starting position in the reference sequence
            - the reference sequence strand
            - the reference sequence id and its number in the sequence file            

Like for FASTA file, `FastqFileReader` and `AlnFileReader` makes it easier to load and work with fastq and aln file

In [ ]:
files_in_tree(p2simreads);

yf
  |--single_69seq_150bp
  |    |--single_69seq_150bp.fq (0)
  |    |--single_69seq_150bp.aln (1)
  |    |--single_69seq_150bp_list_reads.json (2)


In [ ]:
# suffix = '1' if 'paired' in p2simreads.stem else ''
suffix = '2' if 'paired' in p2simreads.stem else ''

p2fastq = p2simreads / f"{p2simreads.stem}{suffix}.fq"
print(p2fastq.name)
assert p2fastq.is_file(), f"{p2fastq} is not a file"
p2aln = p2simreads / f"{p2simreads.stem}{suffix}.aln"
print(p2aln.name)
assert p2aln.is_file(), f"{p2aln} is not a file"

print(f"fq reads file:  {p2fastq.name}\naln reads file: {p2aln.name}")

fastq = FastqFileReader(p2fastq)
assert fastq.re_rule_name is not None
aln = AlnFileReader(p2aln)
assert aln.re_rule_name is not None

single_69seq_150bp.fq
single_69seq_150bp.aln
fq reads file:  single_69seq_150bp.fq
aln reads file: single_69seq_150bp.aln


### Exploring information about the simreads in fastaq

In [ ]:
fastq.re_rule_name, aln.re_header_rule_name

('fastq_art_illumina_ncbi_std', 'aln_art_illumina-refseq-ncbi-std')

It is possible to parse metadata from each simread definition line in one step. 

> Note: depending on the size of the fastq file, this operation may take some time.

In [ ]:
%%time
simread_metadata = fastq.parse_file()
print(f"Total of {len(simread_metadata):,d} simreads parsed")
for i, (k,v) in enumerate(simread_metadata.items(), start=1):
    print(f"Read {i}:")
    pprint(v)
    print()

    if i >= 3: break

Total of 1,160,343 simreads parsed
Read 1:
{'readid': '11089:ncbi:1-17000',
 'readnb': '17000',
 'refseqnb': '1',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}

Read 2:
{'readid': '11089:ncbi:1-16999',
 'readnb': '16999',
 'refseqnb': '1',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}

Read 3:
{'readid': '11089:ncbi:1-16998',
 'readnb': '16998',
 'refseqnb': '1',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}

CPU times: user 50.1 s, sys: 382 ms, total: 50.5 s
Wall time: 51.3 s


### Exploring information in ALN header

The header of ALN file includes:
- the ART command used to simulate the reads
- the list of reference sequences used to create the reads, with metadata

First we can see from which reference fasta file the simulated sequences were imported, by checking the ART command in the alignment file's header. Then we can open a fasta reader for the reference fasta file as well:

In [ ]:
aln.header['command']

'/usr/bin/art_illumina -i /home/vtec/projects/bio/metagentools/data/ncbi/refsequences/yf/yf_2023_yellow_fever.fa -ss HS25 -l 150 -f 250 -o /home/vtec/projects/bio/metagentools/data/ncbi/simreads/yf/single_69seq_150bp/single_69seq_150bp -rs 1731415434'

We see that the simulated reads were generated from the reference sequences in `data/cov_data/cov_virus_sequences_ten.fa`, that is the same as the file in our `fasta`

In [ ]:
fasta.path

PosixPath('/home/vtec/projects/bio/metagentorch/data/ncbi/refsequences/yf/yf_2023_yellow_fever.fa')

Then we can review the metadata available for each reference sequence, as listed in `aln`'s header.

This information can be seen in `aln.header` or with the method `parse_header_reference_sequences()`

In [ ]:
print('\n'.join(aln.header['reference sequences']))

@SQ	11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971	10234
@SQ	11089:ncbi:2	2	U54798	11089	ncbi	Ivory_Coast_1982	10231
@SQ	11089:ncbi:3	3	DQ235229	11089	ncbi	Ethiopia_1961	10234
@SQ	11089:ncbi:4	4	AY572535	11089	ncbi	Gambia_2001	10231
@SQ	11089:ncbi:5	5	MF405338	11089	ncbi	Ghana_Hsapiens_1927	10231
@SQ	11089:ncbi:6	6	U21056	11089	ncbi	Senegal_1927	10231
@SQ	11089:ncbi:7	7	AY968065	11089	ncbi	Uganda_1948	10234
@SQ	11089:ncbi:8	8	JX898871	11089	ncbi	ArD114896_Senegal_1995	10231
@SQ	11089:ncbi:9	9	JX898872	11089	ncbi	Senegal_Aedes-aegypti_1995	10231
@SQ	11089:ncbi:10	10	GQ379163	11089	ncbi	Peru_Hsapiens_2007	10231
@SQ	11089:ncbi:11	11	DQ118157	11089	ncbi	Spain_Vaccine_2004	10231
@SQ	11089:ncbi:12	12	MF289572	11089	ncbi	Singapore_2017	10231
@SQ	11089:ncbi:13	13	KU978764	11089	ncbi	Sudan_Hsapiens_1941	10231
@SQ	11089:ncbi:14	14	JX898878	11089	ncbi	ArD181250_Senegal_2005	10231
@SQ	11089:ncbi:15	15	JX898879	11089	ncbi	ArD181676_Senegal_2005	10231
@SQ	11089:ncbi:16	16	JX898881	11089	ncbi	Senegal

With `AlnFileReader.parse_header_reference_sequences`, we can extract information on all the sequences used to generate the reads:

In [ ]:
header_refseq_meta = aln.parse_header_reference_sequences()

print(f"Reads simulated from {len(header_refseq_meta)} reference sequences.\n")

for i, (k,v) in enumerate(header_refseq_meta.items()):
    print(f"Reference Sequence {i}:")
    pprint(v)
    print()

    if i+1 >= 3: break

Reads simulated from 69 reference sequences.

Reference Sequence 0:
{'organism': 'Angola_1971',
 'refseq_accession': 'AY968064',
 'refseq_length': '10234',
 'refseqid': '11089:ncbi:1',
 'refseqnb': '1',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}

Reference Sequence 1:
{'organism': 'Ivory_Coast_1982',
 'refseq_accession': 'U54798',
 'refseq_length': '10231',
 'refseqid': '11089:ncbi:2',
 'refseqnb': '2',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}

Reference Sequence 2:
{'organism': 'Ethiopia_1961',
 'refseq_accession': 'DQ235229',
 'refseq_length': '10234',
 'refseqid': '11089:ncbi:3',
 'refseqnb': '3',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}



We see that the first reference sequence is a **Coronavirus BtRs-BetaCoV/YN2018D**

### Exploring metadata available for simulated read's

In [ ]:
%%time
reads_meta = aln.parse_file(add_ref_seq_aligned=True, add_read_seq_aligned=True)

print(f"{len(reads_meta):,d} reads were simulated.\n")

for i, (k,v) in enumerate(reads_meta.items()):
    print(f"Read {i}")
    print(k)
    pprint(v)
    print()
    if i+1>=3: break

1,160,343 reads were simulated.

Read 0
11089:ncbi:1-17000
{'aln_start_pos': '3131',
 'read_seq_aligned': 'CACAACAGAGGGATCACTGTGATTGAATTCCAGCCACTGACCAAAAGTATGACCACAGATATGTTCATTTTCATTAAGGGAATTCCTTTGTCCATAAATGAAAGAACTGAGGCAGATTGGGCTATTCCCGATAGTGACAAATTGCCATAC',
 'readid': '11089:ncbi:1-17000',
 'readnb': '17000',
 'ref_seq_aligned': 'CACAACAGAGGGATCACTGTGATTGAATTCCAGCCACTGACCAAAAGTATGACCACAGATATGTTCATTTTCATGAAGGGAATTCCTTTGTCCATAAATGAAAGAACTGAGGCAGATTGGGCTATTCCCGATAGTGACAAATTGCCATAC',
 'refseq_strand': '-',
 'refseqid': '11089:ncbi:1',
 'refseqnb': '1',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}

Read 1
11089:ncbi:1-16999
{'aln_start_pos': '27',
 'read_seq_aligned': 'ACCCTGGGCGTCAATATGGTAAGACGAGGGGTTCGCTCCTTGTCAAACAAAATAAAACAAAAAACAAAACAGATTGGAAACAGACCTGGCCCTTCAAGAGGTGTTCAAGGATTTATTTTCTTCTTTTTGTTTAACATTCTGACTGGGAAA',
 'readid': '11089:ncbi:1-16999',
 'readnb': '16999',
 'ref_seq_aligned': 'ACCCTGGGCGTCAATATGGTAAGACGAGGGGTTCGCTCCTTGTCAAACAAAATAAAACAAAAAACAAAACAGATTGGAAACAGACCTGGCCCTT

### Comparison metadata in fasta, fastq and aln

There is redundancy in metadata across the three sets of files (`.fa`, `.fq`, `.aln`). Let's compare them to confirm this is consistent

Create the following list and dictionaries parsing metadata from fasta, fastq and aln:
- `refseqs_fasta`: metadata in fasta sequence file used as reference sequences to create reads
- `simreads`: metadata on reads, as available in the fastq read file
- `refseqs_aln`: reference sequences with metadata as available in the aln header
- `simread_align`: read sequence and alignment info

In [ ]:
refseqs_fasta = fasta.parse_file(add_seq=True)
simreads = fastq.parse_file(add_readseq=True)
refseqs_aln = aln.ref_sequences
simread_align = aln.parse_file(add_ref_seq_aligned=True, add_read_seq_aligned=True)

Parsing rule is automatically identified as the standard ART Illlumina format, applied to our project standard definition line format for NCBI GenBank sequences. The following shows the parsing rule name and the corresponding regex code:

In [ ]:
print(aln.re_header_rule_name,'\n',aln.re_header_pattern)

aln_art_illumina-refseq-ncbi-std 
 ^@SQ[\t\s]*(?P<refseqid>(?P<reftaxonomyid>\d*):(?P<refsource>\w*):(?P<refseqnb>\d*))[\t\s]*(?P=refseqnb)[\t\s]*(?P<refseq_accession>[\w\d]*)[\t\s]*(?P=reftaxonomyid)[\t\s]*(?P=refsource)[\t\s](?P<organism>.*)[\t\s](?P<refseq_length>\d*)$


In [ ]:
print(f"reads simulated from the following {len(refseqs_fasta)} full sequences from `{fasta.path.name}`:\n")
print('\n'.join([f" {i:02d}: {refseq['accession']:10s} {refseq['organism']}" for i, refseq in enumerate(refseqs_fasta.values(), start=1)]))

reads simulated from the following 69 full sequences from `yf_2023_yellow_fever.fa`:

 01: AY968064   Angola_1971
 02: U54798     Ivory_Coast_1982
 03: DQ235229   Ethiopia_1961
 04: AY572535   Gambia_2001
 05: MF405338   Ghana_Hsapiens_1927
 06: U21056     Senegal_1927
 07: AY968065   Uganda_1948
 08: JX898871   ArD114896_Senegal_1995
 09: JX898872   Senegal_Aedes-aegypti_1995
 10: GQ379163   Peru_Hsapiens_2007
 11: DQ118157   Spain_Vaccine_2004
 12: MF289572   Singapore_2017
 13: KU978764   Sudan_Hsapiens_1941
 14: JX898878   ArD181250_Senegal_2005
 15: JX898879   ArD181676_Senegal_2005
 16: JX898881   Senegal_Aedes_luteocephalus_2005
 17: JX898880   ArD181564_Senegal_2005
 18: JX898877   ArD181464_Senegal_2005
 19: JX898876   Senegal_Aedes_fucifer_2001
 20: KU978765   Guinea_Bissau_Hsapiens_1965
 21: JX898870   Senegal_Ae_fucifer_1996
 22: JX898868   isolate_HD117294_Senegal_1995
 23: JX898875   Senegal_Aedes_fucifer_2000
 24: JX898874   ArD149194_Senegal_2000
 25: JX898873   ArD1492

In [ ]:
print(f"{len(simreads):,d} reads generated and available in fastq file.")
print('For each read, following metadata is available in the fastq file:')
for s in ['readid', 'readnb', 'refseqnb', 'refsource', 'reftaxonomyid', 'readseq']:
    print(f" - {s}")

1,160,343 reads generated and available in fastq file.
For each read, following metadata is available in the fastq file:
 - readid
 - readnb
 - refseqnb
 - refsource
 - reftaxonomyid
 - readseq


In [ ]:
print(f"{len(simread_align):,d} reads generated and available in aln file.")
print('For each read, following metadata is available in the aln file:')
for s in ['aln_start_pos', 'readid', 'readnb', 'refseq_strand', 'refseqid', 'refseqnb', 'refsource', 'reftaxonomyid', 'ref_seq_aligned', 'read_seq_aligned']:
    print(f" - {s}")

1,160,343 reads generated and available in aln file.
For each read, following metadata is available in the aln file:
 - aln_start_pos
 - readid
 - readnb
 - refseq_strand
 - refseqid
 - refseqnb
 - refsource
 - reftaxonomyid
 - ref_seq_aligned
 - read_seq_aligned


Check consistency between refseqs from fasta and from aln 

In [ ]:
# utility functions
def complementary_strand(seq):
    """Converts a strand in its complementary"""
    conv = {'A':'T', 'C':'G', 'G':'C', 'T':'A'}
    return ''.join([conv[base] for base in seq])

strand = 'ATCCGTGGGT'
print(strand, complementary_strand(strand))

def reverse_sequence(seq):
    return seq[::-1]

print(strand, reverse_sequence(strand))

ATCCGTGGGT TAGGCACCCA
ATCCGTGGGT TGGGTGCCTA


### Check aln reference sequence information

In [ ]:
# refseqid = '2591237:ncbi:1'
# refseqid = '11128:ncbi:2'
refseqid = '11089:ncbi:1'


In the source `.fa` file

In [ ]:
original_seq = refseqs_fasta[refseqid]['sequence']
original_seq_accession = refseqs_fasta[refseqid]['accession']

original_seq_accession, len(original_seq)

('AY968064', 10234)

In [ ]:
refseqs_fasta.keys()
refseqs_aln.keys()

dict_keys(['11089:ncbi:1', '11089:ncbi:2', '11089:ncbi:3', '11089:ncbi:4', '11089:ncbi:5', '11089:ncbi:6', '11089:ncbi:7', '11089:ncbi:8', '11089:ncbi:9', '11089:ncbi:10', '11089:ncbi:11', '11089:ncbi:12', '11089:ncbi:13', '11089:ncbi:14', '11089:ncbi:15', '11089:ncbi:16', '11089:ncbi:17', '11089:ncbi:18', '11089:ncbi:19', '11089:ncbi:20', '11089:ncbi:21', '11089:ncbi:22', '11089:ncbi:23', '11089:ncbi:24', '11089:ncbi:25', '11089:ncbi:26', '11089:ncbi:27', '11089:ncbi:28', '11089:ncbi:29', '11089:ncbi:30', '11089:ncbi:31', '11089:ncbi:32', '11089:ncbi:33', '11089:ncbi:34', '11089:ncbi:35', '11089:ncbi:36', '11089:ncbi:37', '11089:ncbi:38', '11089:ncbi:39', '11089:ncbi:40', '11089:ncbi:41', '11089:ncbi:42', '11089:ncbi:43', '11089:ncbi:44', '11089:ncbi:45', '11089:ncbi:46', '11089:ncbi:47', '11089:ncbi:48', '11089:ncbi:49', '11089:ncbi:50', '11089:ncbi:51', '11089:ncbi:52', '11089:ncbi:53', '11089:ncbi:54', '11089:ncbi:55', '11089:ncbi:56', '11089:ncbi:57', '11089:ncbi:58', '11089:ncbi:

In the output`.aln` file

In [ ]:
assert original_seq_accession == refseqs_aln[refseqid]['refseq_accession']
assert len(original_seq) == int(refseqs_aln[refseqid]['refseq_length'])

refseqs_aln[refseqid]['refseq_accession'], int(refseqs_aln[refseqid]['refseq_length'])

('AY968064', 10234)

### Check alignment information

In [ ]:
pprint(simread_align['11089:ncbi:1-13598'])

{'aln_start_pos': '7662',
 'read_seq_aligned': 'AAGGTGGACACGGGAGTGGCCGTGTCGAGAGGGACTGCAAAGCTGAGATGGTTCCACGAGCGTGGCTATGTGAAGCTAGAAGGAAGGGTCACCGATCTTGGGTGTGGACGTGGAGGCTGGTGCTACTATGCAGCAGCTCAAAAAGACGTT',
 'readid': '11089:ncbi:1-13598',
 'readnb': '13598',
 'ref_seq_aligned': 'AAGGTGGACACGGGAGTGGCCGTGTCGAGAGGGACTGCAAAGCTGAGATGGTTCCACGAGCGTGGCTATGTGAAGCTAGAAGGAAGGGTCACCGATCTTGGGTGTGGACGTGGAGGCTGGTGCTACTATGCAGCAGCTCAAAAAGAAGTT',
 'refseq_strand': '+',
 'refseqid': '11089:ncbi:1',
 'refseqnb': '1',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}


Select all reads generated from a single reference sequence

In [ ]:
print(f"Select all reads from reference sequence '{refseqid}''")
reads_from_refseq = {k:v for k,v in simread_align.items() if v['refseqid']==refseqid}
nbr_generated_reads = len(reads_from_refseq)
print(f"Total of {nbr_generated_reads:,d} reads")

Select all reads from reference sequence '11089:ncbi:1''
Total of 17,000 reads


In [ ]:
n = 6
selected_simread = [v for k,v in reads_from_refseq.items()][n]
pprint(selected_simread)

{'aln_start_pos': '3841',
 'read_seq_aligned': 'CATGGCCTCCCCACCCTTCTTTGCCAGGAAATCGGGCAACTCCGTGAGGATCACAAGCATCTCAGCAGCCCCTCTTCTGCCTTCTGCAAACTTGATGAAGTCAGCCAAGGCACTCTGGTCTGAGGAAACTCTCTCGTCACACCATCTAGG',
 'readid': '11089:ncbi:1-16994',
 'readnb': '16994',
 'ref_seq_aligned': 'CATGGCCTCCCCACCCTTCTTTGCCAGGAAATCGGGCAACTCCGTGAGGATCACAAGCATCTCAGCAGCCCCTCTTCTGCCTTCTGCAAACTTGATGAAGTCAGCCAAGGCACTCTGGTCTGAGGAAACTCTCTCGTCACACCATCTAGG',
 'refseq_strand': '-',
 'refseqid': '11089:ncbi:1',
 'refseqnb': '1',
 'refsource': 'ncbi',
 'reftaxonomyid': '11089'}


In [ ]:
def check_alignment(n, reads):
    selected_simread = [v for k,v in reads.items()][n]
    print(f"{'='*80}")
    print(f"checking read {selected_simread['readid']}")
    start = int(selected_simread['aln_start_pos'])
    strand = selected_simread['refseq_strand']
    print(f"simread info:")
    print(f" - from `{strand}` strand")
    print(f" - position: {start:,d}")

    if strand == '+':
        segment_from_refseq = original_seq[start:start+50]
    else:
        segment_from_refseq = complementary_strand(reverse_sequence(original_seq)[start:start+50])

    print('sequences:')
    print(f'- simread seq          :', selected_simread['read_seq_aligned'])
    print(f'- refseq aligned       :', selected_simread['ref_seq_aligned'])
    print(f'- segment in orig. seq :', segment_from_refseq)

In [ ]:
for n in range(nbr_generated_reads-1, nbr_generated_reads-6, -1):
    check_alignment(n, reads_from_refseq)

checking read 11089:ncbi:1-1
simread info:
 - from `-` strand
 - position: 6,520
sequences:
- simread seq          : TGCTCCAAATGCCATTACAAGTCGCTCCCGAGGACTCCACAGTGTGCGCAGCCCAAAGCCCATGAGCACGCCTGGCCGGATGGAAAAGCTGGCAATTAATGCCATGTACATGGCATCTCCTCCGTTGTTAATCTCATGGAAGTGCAATCC
- refseq aligned       : TGCTCCAAATGCCATTACAAGTCGCTCCCGAGGACTCCACAGTGTGCGCAGCCCAAAGCCCATGAGCAGGCCTGGCCGGATGGAAAAGCTGGCAATTAATGCCATGTACATGGCATCTCCTCCGTTGTTAATCTCATGGAAGTGCAATCC
- segment in orig. seq : TGCTCCAAATGCCATTACAAGTCGCTCCCGAGGACTCCACAGTGTGCGCA
checking read 11089:ncbi:1-2
simread info:
 - from `+` strand
 - position: 5,126
sequences:
- simread seq          : ATTAGCACCCACCAGGGTTGTTTTGTCAGAGATGAAAGAAGCTTTTCAGGGGTTGGATGTGAAATTCCACACACAAGCTTTCTCAGCCCATGGGAGTGGGAAGGAGGTCATTGATGCAATGTGCCATGCAACCCTTACATATAGAATGCT
- refseq aligned       : ATTAGCACCCACCAGGGTTGTTTTGTCAGAGATGAAAGAAGCTTTTCAGGGGTTGGATGTGAAATTCCACACACAAGCTTTCTCAGCCCATGGGAGTGGGAAGGAGGTCATTGATGCAATGTGCCATGCAACCCTTACATATAGAATGCT
- segment in orig. seq : ATTAGCACCCA

# end of section